# **Imports**

Using `qutip` tools for calculations and using bessel function from `scipy`.


In [244]:
from qutip import *
from scipy.special import jv

# **Constants**

* `si`: unitary matrix
* `sx`, sy, sz: pauli matrix
* `n`: number of neclei
* `m`: number of electrons
* `extmagfield_m`, `extmagfield_n`: external magnetic fields
* `JJ`, `AA`, `gama`: exchange interaction constants
* `E1`: rescale factor
* `t`: time
* `kappa`: upper limit of chebyshev's expansion

In [245]:
si = qeye(2)

sx = sigmax()

sy = sigmay()

sz = sigmaz()

n = 1

m = 1

extmagfield_m = Qobj([[0, 0, 0],[0, 0, 0],[1, 1, 1]])

extmagfield_n = Qobj([[0, 0, 0],[0, 0, 0],[0.3, 0.3, 0.3]])

JJ = Qobj([[0, 0, 0], [0, 0, 0], [0, 0, 0]])

AA = Qobj([[0, 0, 0] ,[0, 0, 0], [0.99925, 0.625147, 0.551273]])
gama = Qobj([[0.0124425, 0.0806628, 0.00999575],[0.0550028, 0.0758354, 0.07346340],[0.0972069, 0.0723954, 0.07405450]])
E1 = (
    1 / 2 * sum(sum(sum(JJ)))
    + 1 / 2 * sum(sum(sum(gama)))
    + 1 / 2 * sum(sum(sum(AA)))
    + sum(sum(sum(extmagfield_m)))
    + sum(sum(sum(extmagfield_n)))
)
t = 4
tau = E1 * t / 2
kappa = 20

# **Spin Operators**

the function for calculating spin operators.


In [246]:
def spin_op(N):
    Sx = []
    Sy = []
    Sz = []

    for n in range(N):
        x_list = []
        y_list = []
        z_list = []

        for m in range(N):
            x_list.append(si)
            y_list.append(si)
            z_list.append(si)

        x_list[n] = sx
        y_list[n] = sy
        z_list[n] = sz

        Sx.append(tensor(x_list))
        Sy.append(tensor(y_list))
        Sz.append(tensor(z_list))

    return [Sx, Sy, Sz]

# **Hamiltonian**

HS & HB are bare hamiltonians of the central system and bath.

V is the system-bath interaction.

In [247]:
Sx, Sy, Sz = spin_op(m+n)
HS = HB = V = 0
for i in range(m):
    HS += extmagfield_m[0,i] * Sx[i] + extmagfield_m[1,i] * Sy[i] + extmagfield_m[2,i] * Sz[i]
    for j in range(m):
        HS += JJ[0,j] * Sx[i] * Sx[j] + JJ[1,j] * Sy[i] * Sy[j] + JJ[2,j] * Sz[i] * Sz[j]

for i in range(m,m+n):
    HB += extmagfield_n[0,i] * Sx[i] + extmagfield_n[1,i] * Sy[i] + extmagfield_n[2,i] * Sz[i]
    for j in range(m,m+n):
        HB += gama[0,j] * Sx[i] * Sx[j] + gama[1,j] * Sy[i] * Sy[j] + gama[2,j] * Sz[i] * Sz[j]
for i in range(m,m+n):
    for j in range(m):
        V += AA[0,j] * Sx[i] * Sx[j] + AA[1,j] * Sy[i] * Sy[j] + AA[2,j] * Sz[i] * Sz[j]
HH = HS + HB + V


In [248]:
HH

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 2.5281436  0.         0.         0.       ]
 [ 0.        -0.0703564  0.         0.       ]
 [ 0.         0.        -1.4703564  0.       ]
 [ 0.         0.         0.        -0.0718564]]

# **TG**

the function that determines the successive terms in the chebyshev series.

In [249]:
def TG(k, G):
    if k == 0:
        return 1
    elif k == 1:
        return G
    else:
        return 2 * G * TG(k-1, G) - TG(k-2, G)

# **Evolution Operator**

* `G`: is the rescaled hamiltonian
* `jv`: bessel function

In [250]:
G = 2 * HH / E1
UU = 0
for k in range(int(kappa) + 1):
    a = 2
    if k == 0:
        a = 1
    UU += a * ((-1j) ** k) * jv(k,tau) * TG(k, G)



In [251]:
UU

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.77264318+0.63484925j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.96065714+0.2777169j   0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.92037095-0.39103763j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.95897354+0.28347567j]]

# **Test**

creating a final state density matrix using arbitary $\psi_{0}$

if the trace of the final state density matrix is equal to $1$ , the evolution operator is acceptable.

another test is that the product of $U$ and $U^\dag$ should be unitary.


In [252]:
zero = basis(2) 
c = create(2)
one=c*zero
psi=tensor(tensor(one,basis(2)))
psi0=psi*psi.dag()
print("trace: {}".format(psi0.tr())) 
psi0

trace: 1.0


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]

In [253]:
psif = UU * psi0
result = psif * psif.dag()
print("trace: {:.2f}".format(result.tr()))
result

trace: 1.00


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.99999311 0.        ]
 [0.         0.         0.         0.        ]]

In [254]:
UU*UU.dag()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1.00001106 0.         0.         0.        ]
 [0.         0.99998882 0.         0.        ]
 [0.         0.         0.99999311 0.        ]
 [0.         0.         0.         0.9999887 ]]